In [1]:
import numpy as np
import cv2 as cv
import datetime
from sklearn import cluster,datasets
from scipy.stats import multivariate_normal
from sklearn.cluster import KMeans
from datetime import datetime, timedelta

In [2]:
def file_reading(file_path):
    #reading annotations file
    ann_file = open(file_path,"r") #opening file in read mode only
    strings = [x.strip() for x in ann_file.readlines()]
    stimes=[]
    etimes=[]
    for i in range(len(strings)):
        s1,s2=strings[i].split("-")
        stimes.append(s1.strip())
        etimes.append(s2.strip())
    return stimes,etimes

In [3]:
def compare(stime,etime,abnormal_stimes,abnormal_etimes):
    length = len(abnormal_stimes)
    for i in range(length):
        t1 = datetime.strptime(abnormal_stimes[i], '%M:%S').time()
        t2 = datetime.strptime(abnormal_etimes[i], '%M:%S').time()
        obj1 = timedelta(hours=t1.hour, minutes=t1.minute, seconds=t1.second)
        obj2 = timedelta(hours=t2.hour, minutes=t2.minute, seconds=t2.second)
        if (stime >= obj1 and etime <= obj2) or (stime < obj1 and etime > obj1) or (stime < obj2 and etime > obj2):
            return 1
    return -1

In [4]:
class labeling_objects:
    def __init__(self,clip_no,stime,etime,label):
        self.clip_no = clip_no
        self.stime = stime
        self.etime = etime
        self.label = label

In [5]:
def video_input(path,ann_file_path):
    cap = cv.VideoCapture(path)
    ret, frame1 = cap.read()

    #reading first frame
    prvs_gray = cv.cvtColor(frame1,cv.COLOR_BGR2GRAY)
    width=prvs_gray.shape[1]
    height = prvs_gray.shape[0]

    prvs = cv.resize(prvs_gray,(int(width/10),int(height/10)),interpolation = cv.INTER_AREA)

    #intializing all values
    cnt=0
    flow_array = []
    mag_list = []
    flow_array_array = [] #for storing all clips
    secs = 0
    clip = 0
    label_objects_array = []
    abnormal_stimes,abnormal_etimes = file_reading(ann_file_path)
    
    while(True):
        ret, frame2 = cap.read()
        cnt=cnt+1
        if cnt%50 == 0:
            flow_array_array.append(flow_array)
            flow_array=[]
            clip+=1
            #adding labels here after one clip is recorded.
            secs = secs+2
            stime = timedelta(seconds = secs-2)
            etime = timedelta(seconds = secs)
            label = compare(stime,etime,abnormal_stimes,abnormal_etimes)
            label_objects_array.append(labeling_objects(clip,stime,etime,label))
            
        if ret==False:
            break

        #converting frame into gray and resizing 
        gray2 = cv.cvtColor(frame2,cv.COLOR_BGR2GRAY)
        next = cv.resize(gray2,(int(width/10),int(height/10)),interpolation = cv.INTER_AREA)

        #calculating optical flow giving two consecutive frames as input
        flow = cv.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)

        # appending to the flow array
        flow_array.append(flow)

        #changing current frame as previous frame
        prvs = next
    if len(flow_array)!= 0:
        flow_array_array.append(flow_array)
        clip+=1
        secs = secs+1
        stime = timedelta(seconds = secs-1)
        etime = timedelta(seconds = secs)
        label = compare(stime,etime,abnormal_stimes,abnormal_etimes)
        label_objects_array.append(labeling_objects(clip,stime,etime,label))
    flow_array_length = len(flow_array_array)
    print("Total no of clips",flow_array_length)
    print("total frames",cnt)
    
    return flow_array_array,label_objects_array

In [6]:
class GMM:
    
    def __init__(self,X,k,weights,means,variances,n_iter):
        self.X=X
        self.k=k
        self.weights=weights
        self.means = means
        self.variances = variances
        self.eps=1e-8
        self.n_iter = n_iter
        
    def run(self):
        for step in range(self.n_iter):
        
            likelihood=[]
            for j in range(self.k):
                likelihood.append(self.pdf(self.X, self.means[j], np.sqrt(self.variances[j])))
            likelihood = np.array(likelihood)
            
            b = []
            # Maximization step 
            
            for j in range(self.k):
                # use the current values for the parameters to evaluate the posterior
                # probabilities of the data to have been generanted by each gaussian    
                b.append((likelihood[j] * self.weights[j]) / (np.sum([likelihood[i] * self.weights[i] for i in range(self.k)],axis=0))+self.eps)

                # updage mean and variance
                self.means[j] = np.sum(b[j] * self.X) / (np.sum(b[j]+self.eps))
                self.variances[j] = np.sum(b[j] * np.square(self.X - self.means[j])) / (np.sum(b[j]+self.eps))
                
                #print("b dist=",b[j])
                # update the weights
                self.weights[j] = np.mean(b[j])
            #print(self.weights)
            return self

    def pdf(self,data,mean:float,variance:float):
        s1 = 1/(np.sqrt(2*np.pi*variance))
        s2 = np.exp(-(np.square(data - mean)/(2*variance)))
        return s1*s2

In [7]:
#Function for distribution formation from the mag array of a clip
def func_distribution_formation(arr):
    dist_arr =[] 
    height,width,x=arr[0].shape
    for idx in range(int(height)):
        for j in range(int(width)):
            dist=[]
            for i in range(len(arr)):
                dist.append(arr[i][idx,j])
            dist=np.asarray(dist)
            dist_arr.append(dist)
    dist_arr=np.asarray(dist_arr)
    return dist_arr

In [9]:
#intial parameters for em clustering 
def intial_parameters(arr):
    k=4
    arr = np.asarray(arr)
    intial_weights = [(1-0.1)/4 for i in range(k)]
    intial_weights = np.append(intial_weights,0.1) # adding extra weight element for background purpose
    means = np.random.choice(arr.flatten(),(k,2))
    means =np.append(means,[0,0]) # adding extra mean element for background purpose

    cov = np.random.sample(size=k)
    cov = np.append(cov,4.0) # adding extra cov element for background purpose
    return k,intial_weights,means,cov

In [10]:
class data_objects:
    def __init__(self,clip_no,stime,etime,weights_data,label):
        self.clip_no = clip_no
        self.stime = stime
        self.etime = etime
        self.weights_data = weights_data
        self.label = label

In [11]:
#Function to run video input,distribution formation and input to GMM and get updated weights as output
def run(path,ann_file_path):
    mag_arr_all_clips,loa = video_input(path,ann_file_path)
    k,initial_weights,means,cov = intial_parameters(mag_arr_all_clips[0])
    data_object_array=[]
    for index in range(len(mag_arr_all_clips)):
        updated_weights=[]
        updated_means=[]
        updated_variances=[]
        dist_arr = func_distribution_formation(mag_arr_all_clips[index])
        for i in range(dist_arr.shape[0]):
            gmm = GMM(dist_arr[i],k+1,initial_weights.copy(),means.copy(),cov.copy(),50)
            gmm.run()
            updated_weights.append(gmm.weights)
            updated_means.append(gmm.means)
            updated_variances.append(gmm.variances)
        data_object_array.append(data_objects(loa[index].clip_no,loa[index].stime,loa[index].etime,np.asarray(updated_weights).flatten(),loa[index].label))        
    return data_object_array

In [12]:
#running our entire code.
ann_file_path = "E:\\Study\\Sem Project\\Data\\abnormal_times.txt"
path = "E:\\Study\\Sem Project\\Data\\traffic-junction.avi"
total_data_objects = run(path,ann_file_path) #path to video and path to annotations

Total no of clips 1327
total frames 66324


In [13]:
#printing objects value.
print("total no of objects",len(total_data_objects))
for i in range(3):
    print("clip no=",total_data_objects[i].clip_no)
    print("start time=",total_data_objects[i].stime)
    print("end time=",total_data_objects[i].etime)
    print("weights =",total_data_objects[i].weights_data)
    print("label = ",total_data_objects[i].label,"\n")

total no of objects 1327
clip no= 1
start time= 0:00:00
end time= 0:00:02
weights = [0.23775831 0.21021411 0.23205344 ... 0.23086858 0.24774176 0.06312307]
label =  -1 

clip no= 2
start time= 0:00:02
end time= 0:00:04
weights = [0.23794439 0.21014059 0.23198968 ... 0.23168734 0.25006667 0.06248911]
label =  -1 

clip no= 3
start time= 0:00:04
end time= 0:00:06
weights = [0.23807687 0.21008745 0.23194458 ... 0.23159105 0.24978963 0.0625677 ]
label =  -1 



In [14]:
import csv

#writing into the csv file
with open("E:\\Study\\Sem Project\\Data\\weight_motion_vector_data.csv",'w',newline='') as file:
    writer = csv.writer(file)
    for i in range(len(total_data_objects)):
        llist=[]
        llist.append(total_data_objects[i].clip_no)
        llist.append(total_data_objects[i].stime)
        llist.append(total_data_objects[i].etime)
        llist.append(total_data_objects[i].label)
        for j in range(len(total_data_objects[i].weights_data)):
            llist.append(total_data_objects[i].weights_data[j])
        writer.writerow(llist)